E-MAIL SPAM Detection

In [1]:
import nltk

In [2]:
nltk.download_shell()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords'


    Error loading stopwords': Package "stopwords'" not found in index



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords


        C:\Users\akash\AppData\Roaming\nltk_data...
      Unzipping corpora\stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


In [28]:
import numpy as np
import pandas as pd
messages=pd.read_csv('emails.csv')

In [29]:
messages.rename(columns={'text':'message'},inplace=True)
messages.rename(columns={'spam':'label'},inplace=True)

In [30]:
messages.head()

,message,label
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [31]:
messages['label']=messages['label'].map({1:'spam',0:'ham'})

In [33]:
messages['message']=messages['message'].map(lambda x:x[8:])
messages.head()
#stripping off the subject part from message and converting 1,0 to spam ham .

,message,label
0,naturally irresistible your corporate identit...,spam
1,the stock trading gunslinger fanny is merril...,spam
2,unbelievable new homes made easy im wanting ...,spam
3,4 color printing special request additional ...,spam
4,"do not have money , get software cds from her...",spam


In [34]:
#importing stopwords
from nltk.corpus import stopwords
import string

In [37]:
def text_preprocess(mess):
   # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
bow_transformer=CountVectorizer(analyzer=text_preprocess).fit(messages['message'])

In [44]:
messages_bow = bow_transformer.transform(messages['message'])

In [45]:
from sklearn.feature_extraction.text import TfidfTransformer

In [46]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [52]:
print('Shape of Sparse Matrix: ', messages_bow.shape)
print('Amount of Non-Zero occurences: ', messages_bow.nnz)

Shape of Sparse Matrix:  (5728, 37228)
Amount of Non-Zero occurences:  560180


In [56]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)


(5728, 37228)


In [57]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(messages_tfidf, messages['label'])

In [58]:
all_predictions = spam_detect_model.predict(messages_tfidf)
print(all_predictions)

['spam' 'spam' 'spam' ... 'ham' 'ham' 'ham']


In [59]:
from sklearn.metrics import classification_report
print (classification_report(messages['label'], all_predictions))

              precision    recall  f1-score   support

         ham       0.93      1.00      0.97      4360
        spam       1.00      0.77      0.87      1368

    accuracy                           0.95      5728
   macro avg       0.97      0.89      0.92      5728
weighted avg       0.95      0.95      0.94      5728



In [60]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, label_train, label_test = \
train_test_split(messages['message'], messages['label'], test_size=0.2)

In [62]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_preprocess)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [63]:
pipeline.fit(msg_train,label_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_preprocess at 0x000001EFB403B7B8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [64]:
predictions=pipeline.predict(msg_test)

In [65]:
print(classification_report(predictions,label_test))

              precision    recall  f1-score   support

         ham       1.00      0.88      0.94       966
        spam       0.61      1.00      0.76       180

    accuracy                           0.90      1146
   macro avg       0.81      0.94      0.85      1146
weighted avg       0.94      0.90      0.91      1146

